# MCMC2.5: Dense and Sparse Matrices

This section is not necessarily for Marcov chain Monte Carlo itself, but the use of Sparse matrices is important to speed up quantum Monte Carlo simulations.

## Dense matrix

To use dense matrices, `LinearAlgebra` provides wrapper to BLAS/LAPACK.

In [1]:
using LinearAlgebra

I recommend Intel MKL instead of OpenBLAS. You can check whether MKL is used by the following command:

In [2]:
BLAS.vendor()

:mkl

1D Array is called `Vector`.

In [3]:
Array{Int64, 1} == Vector{Int64}

true

2D Array is called `Matrix`, which I later call dense matrix.

In [4]:
Array{Float64, 2} == Matrix{Float64}

true

`Matrix` and `Array` of `Array` are different.

In [5]:
matrix = [1 2
                 3 4]
array =[[1, 2], [3, 4]]
matrix == array

false

Julia's support on 3-rank tensors is limited, but still we can define and use them.

In [6]:
tensor = zeros(Float64, 2, 2, 2)

2×2×2 Array{Float64,3}:
[:, :, 1] =
 0.0  0.0
 0.0  0.0

[:, :, 2] =
 0.0  0.0
 0.0  0.0

Be careful, matrices are stored columnwise.

In [7]:
mat = ones(10000, 10000)
mat[:, 1]
mat[1, :]
@time mat[:, 1];
@time mat[1, :];

  0.000035 seconds (3 allocations: 78.219 KiB)
  0.000309 seconds (3 allocations: 78.219 KiB)


Thus, vertical vectors are more important.

In [8]:
[1.0 2.0; 3.0 4.0] * [5.0; 6.0]

2-element Array{Float64,1}:
 17.0
 39.0

## BLAS/LAPACK

Dense matrices are not memory-efficient, but support BLAS/LAPACK. The most important BLAS operation in quantum Monte Carlo is rank-1 update (or other finite-rank updates).

In [9]:
A = rand(Float64, 1000, 1000)
Ainv = inv(A)

1000×1000 Array{Float64,2}:
  13.5039    15.3539   -3.48992  …  -2.73049    12.3363     29.0417
   7.62503    8.80477  -1.87871     -1.55453     7.05808    16.5354
 -19.3761   -22.0313    5.02901      3.96615   -17.7504    -41.662
  -9.67815  -11.0603    2.45262      2.05864    -8.94929   -20.9152
   4.43783    4.99657  -1.27742     -0.983852    4.09454     9.68627
 -43.1138   -49.3089   10.8197   …   8.75823   -39.6746    -92.8666
   7.94839    9.01022  -2.28172     -1.63392     7.17622    17.212
 -30.9874   -35.3507    7.8061       6.30045   -28.459     -66.6375
 -11.6841   -13.3722    3.04768      2.30612   -10.728     -25.156
  11.9116    13.6195   -3.04198     -2.36742    10.9239     25.7039
 -52.1984   -59.7845   13.1481   …  10.536     -48.0931   -112.588
 -19.9745   -22.7966    4.93357      4.10266   -18.4044    -42.9596
 -13.4706   -15.2518    3.30552      2.7037    -12.3842    -28.7587
   ⋮                             ⋱                        
  29.971     34.1592   -7.85665 

Let's check the Sherman-Morrison formula!
$$\left( A+\vec{u} \vec{v}^T \right)^{-1} = A^{-1} - \frac{A^{-1} \vec{u} \vec{v}^TA^{-1}}{1 + \vec{v}^T A^{-1} \vec{u}}$$
Of course, the vectors are vertical. I define $B = A+\vec{u} \vec{v}^T$

In [10]:
u = rand(Float64, 1000)
v = rand(Float64, 1000)
B = copy(A)
BLAS.ger!(1.0, u, v, B)

1000×1000 Array{Float64,2}:
 1.31962   1.11814   0.504182   1.00876   …  1.02839   0.304281  0.768145
 0.871257  0.386744  1.32125    0.598344     0.914128  0.192313  0.864541
 0.922351  0.693441  1.00169    0.546869     0.657073  0.68525   0.882303
 0.765574  0.545746  0.710324   0.875794     1.49538   0.327563  0.371543
 0.742203  0.209361  1.00277    0.348439     0.541256  0.411337  0.965191
 0.790789  0.554229  1.21141    1.1647    …  1.14102   0.504251  0.571595
 0.957804  0.762132  1.1768     0.873347     0.855631  0.565383  0.336774
 1.25337   0.731997  0.957876   1.24763      1.60365   0.427604  0.523079
 1.20456   1.15702   0.744122   0.991615     1.59652   0.786371  0.616771
 1.42059   0.416556  1.28344    0.912905     1.41753   0.667684  0.396756
 1.37267   0.686352  1.49119    0.29229   …  1.66387   0.841703  1.10397
 0.602746  0.347378  0.0842355  0.711546     0.776959  0.953345  0.927194
 0.961847  1.15944   0.684711   0.542454     1.15371   0.883437  1.07637
 ⋮          

You may need to copy the matrix first because most BLAS operations are destructive. The rank-1 update `BLAS.ger!` is a BLAS-2 function, so it is faster than the BLAS-3 function, matrix inversion. That's why we use the Sherman-Morrison formula to calculate $B^{-1}$ when we already know $A^{-1}$.

In [11]:
Binv = copy(Ainv)
BLAS.ger!(-1.0 / (1.0 + v' * Ainv * u), Ainv * u, (v' * Ainv)', Binv)

1000×1000 Array{Float64,2}:
  0.119966    -0.0163774   -0.21392    …  -0.0230391   0.172599
  0.0437802    0.0983464   -0.0230376      0.0571958   0.182643
 -0.288823    -0.111184     0.356999      -0.124362   -0.490794
 -0.038557     0.00991579   0.0931188     -0.0476346  -0.122525
  0.100855     0.0159234   -0.21585        0.0895744   0.331404
 -0.124445     0.0607231    0.297105   …   0.023855   -0.138314
  0.219782     0.134564    -0.389976       0.0392566   0.541377
 -0.0631598    0.16318      0.236729       0.0978763   0.0661498
 -0.128064    -0.101056     0.219092      -0.0566283  -0.229555
 -0.00690611  -0.0678896   -0.124658      -0.0822608  -0.00441265
 -0.220018    -0.0917433    0.425298   …  -0.0937677  -0.470078
 -0.141938    -0.0205977    0.0791179     -0.0900893  -0.180678
 -0.156635     0.0381804    0.046644      -0.08948    -0.0404019
  ⋮                                    ⋱              
  0.217401    -0.0103242   -0.57381       -0.107767    0.4853
 -0.23254     -0.12

Let's check the results.

In [12]:
norm(B * Binv - I)

1.4955690177227296e-8

Iterating rank-1 updates accumulates some error, so sometimes you have to refresh the updated matrix "from scratch." (Be careful because sometimes it is not really from scratch.)

As for LAPACK, most Julia functions on linear algebra are just wrappers of LAPACK.

In [13]:
eigvals(A)

1000-element Array{Complex{Float64},1}:
  -9.17963412449858 - 0.3967747294588133im
  -9.17963412449858 + 0.3967747294588133im
 -8.884699578737584 - 1.5226368949651186im
 -8.884699578737584 + 1.5226368949651186im
 -8.821210665710549 - 2.549086755691488im
 -8.821210665710549 + 2.549086755691488im
  -8.59176678823733 - 1.244614505677139im
  -8.59176678823733 + 1.244614505677139im
 -8.562123436638599 - 0.7830136033436652im
 -8.562123436638599 + 0.7830136033436652im
 -8.489307268583463 - 3.427655771340261im
 -8.489307268583463 + 3.427655771340261im
  -8.48613583604433 - 0.2064407932532244im
                    ⋮
  8.587061658799243 - 0.685493300392522im
  8.587061658799243 + 0.685493300392522im
  8.747464302084035 - 2.800026342468835im
  8.747464302084035 + 2.800026342468835im
  8.824483310762423 - 2.2780877172214957im
  8.824483310762423 + 2.2780877172214957im
  8.867285298499286 - 1.4480387633889629im
  8.867285298499286 + 1.4480387633889629im
  8.932823301103733 + 0.0im
  9.1301483326399

`eigvals` is just a wrapper for `LAPACK.geevx!`, so you can directely call `LAPACK.geevx!` instead if you wish.

## Sparse matrix

If your program is intensively using sparse matrices, you should use python instead because Julia only supports CSC matrix. Julia's native support for sparse matrices is not strong, so I do not recommend to write a code using multiple types of sparse matrices in Julia.

In [14]:
using SparseArrays

Let's solve a tight-binding model on the 2D square lattice in a poor man's way, i.e. in the real space.

In [15]:
const L = 30
iter1D = 1 : L
nnbondx = zip(Iterators.product(iter1D, iter1D), Iterators.product((mod1(i + 1, L) for i in iter1D), iter1D))
nnbondy = zip(Iterators.product(iter1D, iter1D), Iterators.product(iter1D, (mod1(i + 1, L) for i in iter1D)))
collect(nnbondx), collect(nnbondy)

([((1, 1), (2, 1)) ((1, 2), (2, 2)) … ((1, 29), (2, 29)) ((1, 30), (2, 30)); ((2, 1), (3, 1)) ((2, 2), (3, 2)) … ((2, 29), (3, 29)) ((2, 30), (3, 30)); … ; ((29, 1), (30, 1)) ((29, 2), (30, 2)) … ((29, 29), (30, 29)) ((29, 30), (30, 30)); ((30, 1), (1, 1)) ((30, 2), (1, 2)) … ((30, 29), (1, 29)) ((30, 30), (1, 30))], [((1, 1), (1, 2)) ((1, 2), (1, 3)) … ((1, 29), (1, 30)) ((1, 30), (1, 1)); ((2, 1), (2, 2)) ((2, 2), (2, 3)) … ((2, 29), (2, 30)) ((2, 30), (2, 1)); … ; ((29, 1), (29, 2)) ((29, 2), (29, 3)) … ((29, 29), (29, 30)) ((29, 30), (29, 1)); ((30, 1), (30, 2)) ((30, 2), (30, 3)) … ((30, 29), (30, 30)) ((30, 30), (30, 1))])

These iterators will generate the 2D square lattice.

In [16]:
xytoz(nn::Tuple{Tuple{Int64, Int64}, Tuple{Int64, Int64}}) = (nn[1][2] - 1) * L + nn[1][1], (nn[2][2] - 1) * L + nn[2][1]
nnx = Base.Generator(xytoz, nnbondx)
nny = Base.Generator(xytoz, nnbondy)

Base.Generator{Base.Iterators.Zip{Tuple{Base.Iterators.ProductIterator{Tuple{UnitRange{Int64},UnitRange{Int64}}},Base.Iterators.ProductIterator{Tuple{UnitRange{Int64},Base.Generator{UnitRange{Int64},var"#5#6"}}}}},typeof(xytoz)}(xytoz, Base.Iterators.Zip{Tuple{Base.Iterators.ProductIterator{Tuple{UnitRange{Int64},UnitRange{Int64}}},Base.Iterators.ProductIterator{Tuple{UnitRange{Int64},Base.Generator{UnitRange{Int64},var"#5#6"}}}}}((Base.Iterators.ProductIterator{Tuple{UnitRange{Int64},UnitRange{Int64}}}((1:30, 1:30)), Base.Iterators.ProductIterator{Tuple{UnitRange{Int64},Base.Generator{UnitRange{Int64},var"#5#6"}}}((1:30, Base.Generator{UnitRange{Int64},var"#5#6"}(var"#5#6"(), 1:30))))))

`Base.Generator(f, iter)` is same as `(f(x) for x in iter)`, or you can regard it as a lazy version of `map`, as you saw before.

In [17]:
N = L ^ 2
H = spzeros(Float64, N, N)
for (i, j) in Iterators.flatten((nnx, nny))
    H[i, j] = -1.0
    H[j, i] = -1.0
end
H

900×900 SparseMatrixCSC{Float64,Int64} with 3600 stored entries:
  [2  ,   1]  =  -1.0
  [30 ,   1]  =  -1.0
  [31 ,   1]  =  -1.0
  [871,   1]  =  -1.0
  [1  ,   2]  =  -1.0
  [3  ,   2]  =  -1.0
  [32 ,   2]  =  -1.0
  [872,   2]  =  -1.0
  [2  ,   3]  =  -1.0
  [4  ,   3]  =  -1.0
  [33 ,   3]  =  -1.0
  [873,   3]  =  -1.0
  ⋮
  [898, 897]  =  -1.0
  [28 , 898]  =  -1.0
  [868, 898]  =  -1.0
  [897, 898]  =  -1.0
  [899, 898]  =  -1.0
  [29 , 899]  =  -1.0
  [869, 899]  =  -1.0
  [898, 899]  =  -1.0
  [900, 899]  =  -1.0
  [30 , 900]  =  -1.0
  [870, 900]  =  -1.0
  [871, 900]  =  -1.0
  [899, 900]  =  -1.0

Note that you can rewrite this code by `zip`, but `zip` is not efficient here.

Most of the operations for sparse matrices are similar to the ones for dense matrices. However, sparse arrays are more memory-efficient when the components of the matrix is almost zero. Especially, if the matrix is sparse enough, it significantly reduces the matrix muliplication cost from $O(N^3)$ to $O(N)$.

In [18]:
Hdense = Array(H)
H * H
Hdense * Hdense
@time H * H;
@time Hdense * Hdense;

  0.000179 seconds (8 allocations: 253.766 KiB)
  0.018984 seconds (2 allocations: 6.180 MiB)


`eigvals` does not support sparse matrices, so the calculation of the whole eigenvalues still costs $O(N^3)$. I will discuss this problem later in MCMC5.0.

In [19]:
eigvals(Hdense)

900-element Array{Float64,1}:
 -4.000000000000008
 -3.9562952014676234
 -3.9562952014676207
 -3.9562952014676123
 -3.956295201467606
 -3.912590402935227
 -3.912590402935225
 -3.9125904029352196
 -3.912590402935214
 -3.8270909152852037
 -3.827090915285201
 -3.8270909152852006
 -3.827090915285196
  ⋮
  3.8270909152852006
  3.8270909152852024
  3.827090915285208
  3.9125904029352183
  3.912590402935222
  3.9125904029352276
  3.9125904029352276
  3.9562952014676083
  3.956295201467612
  3.9562952014676123
  3.9562952014676154
  4.000000000000006

For the square (or cubic, etc.) lattice, you can directly begin from a dense matrix. Here's a smart implementation.

In [20]:
H4d = zeros(Float64, L, L, L, L)
for ((i, j), (k, l)) in Iterators.flatten((nnbondx, nnbondy))
    H4d[i, j, k, l] = -1.0
    H4d[k, l, i, j] = -1.0
end
H2d = reshape(H4d, N, N)
eigvals(H2d)

900-element Array{Float64,1}:
 -4.000000000000008
 -3.9562952014676234
 -3.9562952014676207
 -3.9562952014676123
 -3.956295201467606
 -3.912590402935227
 -3.912590402935225
 -3.9125904029352196
 -3.912590402935214
 -3.8270909152852037
 -3.827090915285201
 -3.8270909152852006
 -3.827090915285196
  ⋮
  3.8270909152852006
  3.8270909152852024
  3.827090915285208
  3.9125904029352183
  3.912590402935222
  3.9125904029352276
  3.9125904029352276
  3.9562952014676083
  3.956295201467612
  3.9562952014676123
  3.9562952014676154
  4.000000000000006

## Block checkerboard decomposition/approximation

It is sometimes very useful to approximate a dense matrix by a product of sparse matrices. In the physical models like tight-binding models, block checkerboard docomposition will be a good approximation.

~ under construction ~

## Iterative solvers

Iterative solvers, especially conjugate gradient methods are important for hybrid Monte Carlo simulations for lattice gauge theories.

### Conjugate gradient (CG) method

~ under construction ~

### Preconditioners

FYI a careful choice of a preconditiner is necessary for ill-conditioned matrices, i.e. matrices with a large condition number. I personally recommend the incomplete Cholesky preconditioner in Preconditioners.jl.